In [1]:
import pandas as pd
import os.path
import datetime
import BoW
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords

In [2]:
# Data Loading
path = os.getcwd()
parent_folder, current_folder = os.path.split(path)
df = pd.read_csv(parent_folder + '/0.Raw_data/train/Combined_News_DJIA_train.csv')   # please check if Training data is in the same location on your PC

In [3]:
def cleaning(mess):
    """
    Takes in a string of text, then performs the following:
    1. Lower case of all words
    2. Remove all punctuation
    3. Remove all stopwords
    4. Returns a list of the cleaned text
    """

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)

    # Now just remove any stopwords
    return [word for word in nopunc.split() if word not in stopwords.words('english')]

In [4]:
# Data Cleaning and merging all headlines to one single column

df.iloc[:,2:27] = df.iloc[:,2:27].applymap(str)

# replace the b' and b" which are in the beginning of some headlines
df.iloc[:,2:27] = df.iloc[:,2:27].replace(regex="b'",value="")
df.iloc[:,2:27] = df.iloc[:,2:27].replace(regex='b"',value='')
df.iloc[:,2:27] = df.iloc[:,2:27].apply(lambda x: x.astype(str).str.lower())

#df1['lengths'] = df1['headlines'].apply(len)
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
for i in range(2,27):
    df.iloc[:,i] = df.iloc[:,i].apply(cleaning)

In [10]:
df
#df.to_csv('cleaned_tokenized.csv')

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0.0,"[georgia, downs, two, russian, warplanes, coun...",[nan],"[russia, today, columns, troops, roll, south, ...","[russian, tanks, moving, towards, capital, sou...","[afghan, children, raped, impunity, un, offici...","[150, russian, tanks, entered, south, ossetia,...","[breaking, georgia, invades, south, ossetia, r...","[enemy, combatent, trials, nothing, sham, sali...",...,"[georgia, invades, south, ossetia, russia, get...","[alqaeda, faces, islamist, backlash]","[condoleezza, rice, us, would, act, prevent, i...","[busy, day, european, union, approved, new, sa...","[georgia, withdraw, 1000, soldiers, iraq, help...","[pentagon, thinks, attacking, iran, bad, idea,...","[caucasus, crisis, georgia, invades, south, os...","[indian, shoe, manufactory, series, like, work]","[visitors, suffering, mental, illnesses, banne...","[help, mexicos, kidnapping, surge]"
1,2008-08-11,1.0,"[wont, america, nato, help, us, wont, help, us...","[bush, puts, foot, georgian, conflict]","[jewish, georgian, minister, thanks, israeli, ...","[georgian, army, flees, disarray, russians, ad...","[olympic, opening, ceremony, fireworks, faked]","[mossad, fraudulent, new, zealand, passports, ...","[russia, angered, israeli, military, sale, geo...","[american, citizen, living, sossetia, blames, ...",...,"[israel, us, behind, georgian, aggression]","[believe, tv, neither, russian, georgian, much...","[riots, still, going, montreal, canada, police...","[china, overtake, us, largest, manufacturer]","[war, south, ossetia, pics]","[israeli, physicians, group, condemns, state, ...","[russia, beaten, united, states, head, peak, oil]","[perhaps, question, georgia, russia, conflict]","[russia, much, better, war]",[nan]
2,2008-08-12,0.0,"[remember, adorable, 9yearold, sang, opening, ...","[russia, ends, georgia, operation]","[sexual, harassment, would, children]","[alqaeda, losing, support, iraq, brutal, crack...","[ceasefire, georgia, putin, outmaneuvers, west]","[microsoft, intel, tried, kill, xo, 100, laptop]","[stratfor, russogeorgian, war, balance, power]","[im, trying, get, sense, whole, georgiarussia,...",...,"[us, troops, still, georgia, know, georgia, fi...","[russias, response, georgia, right]","[gorbachev, accuses, us, making, serious, blun...","[russia, georgia, nato, cold, war, two]","[remember, adorable, 62yearold, led, country, ...","[war, georgia, israeli, connection]","[signs, point, us, encouraging, georgia, invad...","[christopher, king, argues, us, nato, behind, ...","[america, new, mexico]","[bbc, news, asiapacific, extinction, man, clim..."
3,2008-08-13,0.0,"[us, refuses, israel, weapons, attack, iran, r...","[president, ordered, attack, tskhinvali, capit...","[israel, clears, troops, killed, reuters, came...","[britains, policy, tough, drugs, pointless, sa...","[body, 14, year, old, found, trunk, latest, ra...","[china, moved, 10, million, quake, survivors, ...","[bush, announces, operation, get, russias, gri...","[russian, forces, sink, georgian, ships]",...,"[elephants, extinct, 2020]","[us, humanitarian, missions, soon, georgia, ru...","[georgias, ddos, came, us, sources]",[nan],"[israeli, defence, minister, us, strike, iran]","[gorbachev, choice]","[witness, russian, forces, head, towards, tbil...","[quarter, russians, blame, us, conflict, poll]","[georgian, president, says, us, military, take...","[2006, nobel, laureate, aleksander, solzhenits..."
4,2008-08-14,1.0,"[experts, admit, legalise, drugs]","[war, south, osetia, 89, pictures, made, russi...","[swedish, wrestler, ara, abrahamian, throws, a...","[russia, exaggerated, death, toll, south, osse...","[missile, killed, 9, inside, pakistan, may, la...","[rushdie, condemns, random, houses, refusal, p...","[poland, us, agree, missle, defense, deal, int...","[russians, conquer, tblisi, bet, seriously, bet]",...,"[bank, analyst, forecast, georgian, crisis, 2,...","[geo

In [7]:
X_train, X_val, y_train, y_val = train_test_split(df.loc[:, df.columns != 'Label'], df['Label'], test_size=0.20, random_state=42)

In [8]:
from gensim.models import Word2Vec

In [74]:
sentences = []
for i in range(1,26):
    sentences.extend(X_train.iloc[:,i].tolist())
    
len(sentences)

31675

In [75]:
model = Word2Vec(sentences, min_count=4)

In [76]:
print(model)

Word2Vec(vocab=10461, size=100, alpha=0.025)


In [79]:
print(model['leave'])

[-0.20441225 -0.36507687 -0.49798822  0.32361165 -0.306292   -0.04944353
 -0.13995168  0.02558778  0.13239343  0.13787332 -0.11129354  0.03404105
  0.1342655   0.01495843 -0.14439295 -0.19214235 -0.6159852   0.24979064
 -0.3555312  -0.3706475  -0.0842519  -0.16175325 -0.20490283  0.11673517
  0.09701269 -0.1794813  -0.02798332  0.12928282  0.01169199  0.19840942
 -0.20286393 -0.08329935  0.513237    0.47349215  0.11655321  0.50718635
 -0.80335075  0.07352114 -0.01929052  0.49953943  0.01287543  0.14084545
 -0.4655705   0.63006324 -0.1037948  -0.34421492 -0.28629658  0.23274681
 -0.17691216  0.3983252   0.3419298  -0.43212914  0.17412412  0.5352121
 -0.06591272 -0.07175014 -0.35434872  0.07836986 -0.6571546   0.23156862
 -0.502617   -0.31446737  0.5022699  -0.3749558  -0.13550384  0.4056973
  0.46915817  0.56225866  0.5430109   0.25763196 -0.44691563 -0.21231283
  0.0562297  -0.5046653   0.02119363 -0.13121101 -0.24890131  0.00448127
 -0.03661867 -0.435935    0.18711187  0.3010982   0.2

/Users/jamoth/anaconda3/envs/comp2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [83]:
model.wv.most_similar(positive='woman')

[('young', 0.9963662624359131),
 ('men', 0.9962849020957947),
 ('death', 0.995673656463623),
 ('sentenced', 0.9951450228691101),
 ('arrested', 0.9949872493743896),
 ('five', 0.9941661953926086),
 ('protesters', 0.9932349324226379),
 ('jail', 0.992759108543396),
 ('girl', 0.9923790693283081),
 ('three', 0.9922330379486084)]

In [84]:
sentences

[['leave', 'damn', 'engineer'],
 ['technologically',
  'illiterate',
  'mp',
  'masterminded',
  'uk',
  'porn',
  'blocker',
  'get',
  'hacked',
  'threatens',
  'reporter',
  'writing'],
 ['turkish',
  'police',
  'started',
  'arresting',
  'people',
  'tweeting',
  'facebooking',
  'protests',
  'local',
  'news',
  'cnn',
  'turk',
  'needs',
  'translation'],
 ['stolen', 'maple', 'syrup', 'recovered'],
 ['church',
  'scientology',
  'wont',
  'dissolved',
  'france',
  'due',
  'typo',
  'obscure',
  'law',
  'voted',
  'two',
  'weeks',
  'church',
  'went',
  'trial',
  'explicitly',
  'prevents'],
 ['ecuador',
  'president',
  'blasts',
  'coverage',
  'snowden',
  'case',
  'theyve',
  'managed',
  'focus',
  'attention',
  'snowden',
  'wicked',
  'countries',
  'support',
  'making',
  'us',
  'forget',
  'terrible',
  'things',
  'us',
  'people',
  'whole',
  'world',
  'denounced'],
 ['india',
  'girl',
  'ends',
  'child',
  'marriage',
  'young',
  'woman',
  'child',